# Práctico 5: Aprendizaje No Supervisado

## Diversificación
Cuando se construyen carteras de inversión, se busca obtener buenos retornos mientras se limita la variabilidad de los retornos sobre el tiempo. Una forma de conseguir esto es elegir acciones que muestren diferentes "patrones de retornos". Para eso, es necesario agrupar a las acciones que tengan patrones similares en clusters y seleccionar nuestras inversiones a partir de ellos.

Una agrupación de acciones "natural" viene a partir de la Industria a la cual pertenecen, algo con lo que trabajamos en el Práctico 2 para aquellas que pertenecen al índice S&P500. Consigamos esos datos nuevamente.

In [1]:
!pip install yahoo_fin --upgrade

     |████████████████████████████████| 81 kB 451 kB/s eta 0:00:01
     |████████████████████████████████| 83 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 2.8 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=b7cf89e0b26f03722f1c918f2fe47946ef65c443b57c4bf63bd704951312b61d
  Stored in directory: /Users/ariel/Library/Caches/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13486 sha256=38cf3639e6a5eed54498566c7c690ba315f3b56ea77d6779d49da06877f30532
  Stored in directory: /Users/ariel/Library/Caches/pip/wheels/a0/b8/b7/8c942b2c5be5158b874a88195116b05ad124bac795f6665e65
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24581 sha256=65807c5a023a93fa6e27754ab89

In [20]:
!pip install yahoofinancials

  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15190 sha256=0e289412e043be61cedfe44fc24c66957ea988c0fc8c4e39f0e1e6aa4d9fa13a
  Stored in directory: /Users/ariel/Library/Caches/pip/wheels/6a/90/0c/08c7ac2ce60b9ac91529417d471e59244f9f96848c86f14809
Successfully built yahoofinancials


In [21]:
import pandas as pd
from pandas_datareader import data
from yahoo_fin.stock_info import *
from datetime import datetime
import yfinance as yf
from yahoofinancials import YahooFinancials

In [3]:
# lista de constituyentes del S&P500 desde Wikipedia

wiki = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
sp500 = wiki[0] # el primer DataFrame corresponde a los constituyentes del S&P500
sp500.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [4]:
# Análisis de sectores
n_sect = sp500['GICS Sector'].unique().shape # cantidad de sectores únicos
acciones_sect = sp500['GICS Sector'].value_counts() # cantidad de acciones por sector

print('Cantidad de sectores únicos: ', n_sect[0])
print('Cantidad de acciones por sector: ', acciones_sect)

Cantidad de sectores únicos:  11
Cantidad de acciones por sector:  Industrials               74
Information Technology    74
Financials                65
Health Care               64
Consumer Discretionary    63
Consumer Staples          32
Real Estate               29
Utilities                 28
Materials                 28
Communication Services    27
Energy                    21
Name: GICS Sector, dtype: int64


## Clustering Jerárquico

Realizar clustering para series de tiempo es un poco diferente a hacerlo con datos tabulares (no tanto, algunas técnicas sirven para ambos sin problemas), pero existen herramientas para poder realizarlo en SKLearn. En este caso, nos vamos a meter con el **clustering jerárquico**: vamos a generar *clusters anidados* al juntarlos o separarlos de manera sucesiva. La jerarquía se representa como un árbol o *dendrograma*, donde la raíz es el único cluster que junta a todas las acciones y las hojas representan a los clusters formados por una sola acción. Como siempre, una buena lectura para este tipo de conceptos puede ser [Wikipedia](https://en.wikipedia.org/wiki/Hierarchical_clustering)

<p><a href="https://commons.wikimedia.org/wiki/File:Hierarchical_clustering_simple_diagram.svg#/media/File:Hierarchical_clustering_simple_diagram.svg"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/ad/Hierarchical_clustering_simple_diagram.svg/1200px-Hierarchical_clustering_simple_diagram.svg.png" alt="Hierarchical clustering simple diagram.svg", width=300></a><br>

Es posible hacer clustering jerárquico con la función con SKLearn utilizando la función [AgglomerativeClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html), que realiza por nosotros este proceso, a partir de clusters de una sola acción y luego los va uniendo a partir de alguna función de *linkage*, que es algún tipo de distancia entre ellos.

Nuestro primer objetivo va a ser intentar recuperar las industrias a las cuales pertenecen las acciones basándonos sólo en sus patrones de **retornos**:

- Elijamos 4 sectores del S&P500 y consigamos los precios de sus acciones (o algunas de ellas si son muchas) durante todo el 2020.
- Obtengamos los retornos diarios de las acciones y apliquemos la función de clustering a ellos.
- Generemos la visualización del dendrograma (se puede seguir el [siguiente tutorial](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py))
- Veamos si coinciden algunas de las posibles agrupaciones de acciones con las industrias a las que pertenecen y saquemos algunas conclusiones. Se puede utilizar diferentes funciones de *linkage* para ver si alguna consigue mejores agrupaciones.
    

In [5]:
# Elegimos 4 sectores del S&P500: Information Technology, Financials, Utilities y Energy

sector = ['Information Technology', 'Financials', 'Utilities', 'Energy']

spy = sp500.loc[sp500['GICS Sector'].isin(sector)]

tickers_companias = list(spy['Symbol'])

print(tickers_companias)
len(tickers_companias)

['ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'AKAM', 'LNT', 'ALL', 'AEE', 'AEP', 'AXP', 'AIG', 'AWK', 'AMP', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'ANET', 'AJG', 'AIZ', 'ATO', 'ADSK', 'ADP', 'BKR', 'BAC', 'BRK.B', 'BLK', 'BK', 'AVGO', 'BR', 'BRO', 'CTRA', 'CDNS', 'COF', 'CBOE', 'CDW', 'CNP', 'CDAY', 'SCHW', 'CVX', 'CB', 'CINF', 'CSCO', 'C', 'CFG', 'CTXS', 'CME', 'CMS', 'CTSH', 'CMA', 'COP', 'ED', 'GLW', 'DVN', 'FANG', 'DFS', 'D', 'DTE', 'DUK', 'DXC', 'EIX', 'ENPH', 'ETR', 'EOG', 'EVRG', 'ES', 'RE', 'EXC', 'XOM', 'FFIV', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FTNT', 'BEN', 'IT', 'GL', 'GPN', 'GS', 'HAL', 'HIG', 'HES', 'HPE', 'HPQ', 'HBAN', 'INTC', 'ICE', 'IBM', 'INTU', 'IVZ', 'IPGP', 'JKHY', 'JPM', 'JNPR', 'KEY', 'KEYS', 'KMI', 'KLAC', 'LRCX', 'LNC', 'L', 'MTB', 'MRO', 'MPC', 'MKTX', 'MMC', 'MA', 'MET', 'MCHP', 'MU', 'MSFT', 'MPWR', 'MCO', 'MS', 'MSI', 'MSCI', 'NDAQ', 'NTAP', 'NEE', 'NI', 'NTRS', 'NLOK', 'NRG', 'NVDA', 'NXPI', 'OXY', 'OKE', 'ORCL', 'PAYX', 'PAYC', 'PYPL', 'PB

188

In [13]:
# convierto mi lista en un string
string_tickers = ' '.join(tickers_companias)

In [23]:
yf.pdr_override()

# download dataframe
start_date = "2020-01-01"
end_date = "2020-12-31"

df = data.get_data_yahoo(string_tickers, start=start_date, end=end_date)

df

[*********************100%***********************]  188 of 188 completed

1 Failed download:
- BRK.B: No data found, symbol may be delisted


Adj Close                                                  \
                  AAPL         ACN        ADBE         ADI         ADP   
Date                                                                     
2019-12-31         NaN         NaN         NaN         NaN         NaN   
2020-01-02   74.096458  204.910965  334.429993  116.506660  163.903595   
2020-01-03   73.376083  204.569702  331.809998  114.455727  163.557129   
2020-01-06   73.960762  203.233841  333.709991  113.111023  163.778473   
2020-01-07   73.612923  198.846008  333.390015  115.684357  161.796082   
...                ...         ...         ...         ...         ...   
2020-12-23  130.347565  255.051727  496.910004  141.411163  172.690750   
2020-12-24  131.352844  254.913025  499.859985  142.645370  174.041214   
2020-12-28  136.050781  257.270966  498.950012  142.852722  174.090500   
2020-12-29  134.239273  255.368790  502.109985  141.845612  172.375320   
2020-12-30  133.094666  255.150818  497.450012  144.333771  171.330444   

                                                                    ...  \
                  ADSK        AEE        AEP        AES        AFL  ...   
Date                                                                ...   
2019-12-31         NaN        NaN        NaN        NaN        NaN  ...   
2020-01-02  187.830002  72.665764  88.182365  19.014515  50.947163  ...   
2020-01-03  184.949997  72.694435  88.088020  18.796177  50.593822  ...   
2020-01-06  187.119995  72.761314  88.380516  19.014515  50.450581  ...   
2020-01-07  187.500000  73.028854  88.399391  19.080967  49.973103  ...   
...                ...        ...        ...        ...        ...  ...   
2020-12-23  300.179993  74.212563  78.644043  23.065132  42.724495  ...   
2020-12-24  301.390015  74.947624  79.335869  22.849113  43.028763  ...   
2020-12-28  299.649994  75.222054  79.199455  22.878569  43.107277  ...   
2020-12-29  298.109985  75.045631  79.102013  22.888388  42.940426  ...   
2020-12-30  299.410004  75.437675  79.423569  22.367975  43.195610  ...   

                Volume                                                         \
                   WFC       WLTW         WMB       WRB         WU        XEL   
Date                                                                            
2019-12-31         NaN        NaN         NaN       NaN        NaN        NaN   
2020-01-02  16803100.0   465800.0   6970000.0  873500.0  8124900.0  2915400.0   
2020-01-03  15608800.0   411400.0   7542700.0  438400.0  4993800.0  2422500.0   
2020-01-06  13200300.0   620200.0  12104200.0  650500.0  8018800.0  2217100.0   
2020-01-07  13278600.0   443200.0   8952900.0  686200.0  6180600.0  1861100.0   
...                ...        ...         ...       ...        ...        ...   
2020-12-23  38383700.0  1233400.0   6606700.0  457200.0  2486900.0  1153200.0   
2020-12-24  14443700.0   293900.0   2684800.0  149500.0   814300.0   598400.0   
2020-12-28  21658500.0   934300.0   6105700.0  539300.0  2446000.0  1799800.0   
2020-12-29  19584400.0   399400.0   6035800.0  398000.0  2400900.0  1761400.0   
2020-12-30  18519300.0   692200.0   6792400.0  365700.0  1555700.0  1296400.0   

                                                        
                 XLNX         XOM      ZBRA       ZION  
Date                                                    
2019-12-31        NaN         NaN       NaN        NaN  
2020-01-02  3234100.0  12456400.0  387800.0  1528700.0  
2020-01-03  3521900.0  17386900.0  305300.0  1215800.0  
2020-01-06  2832700.0  20081900.0  322600.0  1217500.0  
2020-01-07  3050000.0  17387700.0  236300.0  1450900.0  
...               ...         ...       ...        ...  
2020-12-23  1766400.0  19085900.0  191100.0   706800.0  
2020-12-24  1167500.0   8039000.0  138300.0   327700.0  
2020-12-28  1978500.0  23877500.0  251200.0   835600.0  
2020-12-29  1489900.0  20287700.0  185500.0  1084100.0  
2020-12-30  2892600.0  23807300.0  166100.0   728

## Probemos los clusters!

Una vez que se haya podido realizar el clustering y estemos "conformes", elijamos un nivel de linkage que nos permita tener 4 clusters y construyamos la siguiente estrategia de inversión:
- Elegir una acción de cada cluster aleatoriamente
- Invertir el 25% del dinero en cada una de ellas
- La estrategia corre durante todo 2021 hasta finales de agosto
- Comenzar con $10.000

Grafiquemos el avance de la misma, junto al avance del S&P500 y calculemos el Sharpe Ratio de cada una de ellas.

## Agregado Opcional

Si disponemos de más tiempo, podemos intentar elegir nuestras acciones desde los clusters de manera más inteligente. Por ejemplo, en vez de hacerlo de manera aleatoria, podemos elegir la que tenga la menor volatilidad, o la que tenga el mejor rendimiento predecible según algún indicador técnico o modelo que hayamos podido entrenar en los prácticos 3 y 4.

También podemos hacer que la estrategia cambie de acciones por día o mes de acuerdo al criterio que se elija. Graficar nuevamente estas estrategias y compararlas con el índice.